In [2]:
!ls


Carmen_data_cleaning.ipynb             josh_GOT_Linear.ipynb
Carmen_data_cleaningv2.ipynb           josh_GOT_Machine.ipynb
PCA_carmen.ipynb                       rui_genAI.ipynb
clustering_kmeans_carmen.ipynb         rui_gnn.ipynb
franzi_exploring_houses.ipynb          rui_openAI_images.ipynb
franzi_initial_cluster_exploring.ipynb rui_scraped_data.ipynb
franzi_trying_stories.ipynb            rui_scraped_deaths.ipynb
josh_GOT_FEAT.ipynb                    rui_testing_images_prompt.ipynb


In [9]:
!pip install prince


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.6/415.6 kB 8.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.9/996.9 kB 10.7 MB/s eta 0:00:0000:0100:01
  Using cached toolz-0.12.0-py3-none-any.whl (55 kB)

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


# Import packages

In [10]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from scipy import sparse

import prince


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler


# Load data

In [6]:
file_path = '../processed_data/cleaned_data_final.csv'


In [91]:
df = pd.read_csv(file_path)

df.head()


,name,male,origin,isMarried,isNoble,popularity,isAlive,season,episode,deaths,episode_num
0,Viserys II Targaryen,1,House Targaryen,0,0,0.605351,0,8,The Bells,Lord Varys - Burned alive by Drogon with his f...,71
1,Walder Frey,1,House Frey,1,1,0.896321,0,6,The Winds of Winter,Lord Walder Frey - Throat slit by Arya Stark w...,60
2,Addison Hill,1,House Lannister,0,1,0.267559,1,0,NaN,NaN,0
3,Aemma Arryn,0,House Arryn,1,1,0.183946,0,5,The Gift,Maester Aemon Targaryen - Died of natural causes.,47
4,Sylva Santagar,0,House Martell,1,1,0.043478,1,0,NaN,NaN,0


In [92]:
df[df.name.duplicated(keep=False)]


,name,male,origin,isMarried,isNoble,popularity,isAlive,season,episode,deaths,episode_num
1658,Beric Dondarrion,1,House Tully,0,1,0.434783,0,8,The Iron Throne,Lord Beric Dondarrion - 22 (Deceased),72
1659,Beric Dondarrion,1,Peasant,0,1,0.434783,0,8,The Iron Throne,Lord Beric Dondarrion - 22 (Deceased),72
1660,Beric Dondarrion,1,Night's Watch,0,1,0.434783,0,8,The Iron Throne,Lord Beric Dondarrion - 22 (Deceased),72
1661,Beric Dondarrion,1,Dothraki,0,1,0.434783,0,8,The Iron Throne,Lord Beric Dondarrion - 22 (Deceased),72


In [93]:
df.drop_duplicates(subset='name',inplace=True)


In [94]:
df[df.name.duplicated(keep=False)]


,name,male,origin,isMarried,isNoble,popularity,isAlive,season,episode,deaths,episode_num


In [77]:
df.drop(labels=['episode','deaths'],axis=1, inplace=True)
df.set_index(keys='name', drop=True, inplace=True)


In [78]:
# Categorical features --> OneHotEncoder
# X_cat = df.select_dtypes(include = ['object']).columns.tolist()
# Features that do not need transformation
no_transformation_features = ['male', 'isMarried','isNoble','isAlive', 'origin']
# Ordinal features --> MinMax Scaler
X_ord = [col for col in df.columns if col not in no_transformation_features and col not in X_cat]

print(f'X_cat: {X_cat}, X_ord:{X_ord}, features without transformation: {no_transformation_features}')


X_cat: ['origin'], X_ord:['popularity', 'season', 'episode_num'], features without transformation: ['male', 'isMarried', 'isNoble', 'isAlive', 'origin']


In [79]:
# Impute then scale numerical values:
num_transformer = Pipeline(steps=[
    ('MinMax_scaler', MinMaxScaler())
])



# Parallelize "num_transformer" and "cat_transfomer"
preproc_baseline = ColumnTransformer(
    transformers = [
        ('num_transformer', num_transformer, X_ord),
        ('passthrough', 'passthrough', no_transformation_features)
    ]
)


In [80]:
processed_df = preproc_baseline.fit_transform(df)


In [81]:
processed_df


array([[0.605351170568561, 1.0, 0.986111111111111, ..., 0, 0,
        'House Targaryen'],
       [0.896321070234113, 0.75, 0.8333333333333333, ..., 1, 0,
        'House Frey'],
       [0.267558528428093, 0.0, 0.0, ..., 1, 1, 'House Lannister'],
       ...,
       [0.0301003344481605, 0.0, 0.0, ..., 0, 1, 'Dothraki'],
       [0.130434782608695, 0.0, 0.0, ..., 0, 1, 'Peasant'],
       [0.210702341137123, 1.0, 1.0, ..., 1, 0, 'House Lannister']],
      dtype=object)

In [82]:
# Get the column names for the columns that were transformed
# 'num_transformer' is a Pipeline, so we need to get the feature names from the last step (MinMaxScaler in this case)
transformed_columns = preproc_baseline.named_transformers_['num_transformer'].get_feature_names_out(X_ord)

# Combine with the columns that were passed through
final_column_names = list(transformed_columns) + no_transformation_features

# Convert the NumPy array to a DataFrame with the correct column names
processed_df = pd.DataFrame(processed_df, columns=final_column_names)

# If the original DataFrame (df) had an index, set it to the new DataFrame
if df.index is not None:
    processed_df.index = df.index
    #processed_df.set_index(keys='name', drop=True, inplace=True)


In [95]:
processed_df.columns


Index(['popularity', 'season', 'episode_num', 'male', 'isMarried', 'isNoble',
       'isAlive', 'origin'],
      dtype='object')

In [96]:
processed_df = processed_df.reindex(columns=['origin', 'popularity', 'season', 'episode_num', 'male', 'isMarried', 'isNoble',
       'isAlive'])


In [103]:
processed_df


,origin,popularity,season,episode_num,male,isMarried,isNoble,isAlive
name,,,,,,,,
Viserys II Targaryen,House Targaryen,0.605351,1.0,0.986111,1,0,0,0
Walder Frey,House Frey,0.896321,0.75,0.833333,1,1,1,0
Addison Hill,House Lannister,0.267559,0.0,0.0,1,0,1,1
Aemma Arryn,House Arryn,0.183946,0.625,0.652778,0,1,1,0
Sylva Santagar,House Martell,0.043478,0.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...,...
Luwin,House Stark,0.351171,0.25,0.277778,1,0,0,0
Reek,House Bolton,0.09699,0.375,0.402778,1,0,0,0
Symeon Star-Eyes,Dothraki,0.0301,0.0,0.0,1,0,0,1


In [110]:
processed_df= processed_df.infer_objects()


In [111]:
processed_df.dtypes


origin          object
popularity     float64
season         float64
episode_num    float64
male             int64
isMarried        int64
isNoble          int64
isAlive          int64
dtype: object

In [112]:
processed_df


,origin,popularity,season,episode_num,male,isMarried,isNoble,isAlive
name,,,,,,,,
Viserys II Targaryen,House Targaryen,0.605351,1.000,0.986111,1,0,0,0
Walder Frey,House Frey,0.896321,0.750,0.833333,1,1,1,0
Addison Hill,House Lannister,0.267559,0.000,0.000000,1,0,1,1
Aemma Arryn,House Arryn,0.183946,0.625,0.652778,0,1,1,0
Sylva Santagar,House Martell,0.043478,0.000,0.000000,0,1,1,1
...,...,...,...,...,...,...,...,...
Luwin,House Stark,0.351171,0.250,0.277778,1,0,0,0
Reek,House Bolton,0.096990,0.375,0.402778,1,0,0,0
Symeon Star-Eyes,Dothraki,0.030100,0.000,0.000000,1,0,0,1


#### Fitting

In [129]:
famd = prince.FAMD(
    n_components=8,
    n_iter=10,
    copy=True,
    check_input=True,
    random_state=42,
    engine="sklearn",
    handle_unknown="error"  # same parameter as sklearn.preprocessing.OneHotEncoder
)


In [130]:
famd = famd.fit(processed_df)


/Users/Carmen/.pyenv/versions/3.10.6/envs/game-of-thrones-survival/lib/python3.10/site-packages/prince/pca.py:175: PerformanceWarning: Concatenating sparse arrays with multiple fill values: '[-0.8428710783247098, -1.3377474893729913, -1.4647274544877327, -0.5954607326011404, -1.1608773306309617, -1.0734821019593819, -0.9756806462851751, -1.2442054800010094, -0.2150551448197962, -0.25344491871855485, -0.3602129095201821, -0.3362326801646274, -0.3826934349571483, -0.14778251290408706, -0.42711296887461037, -0.4345675101331315, -0.4195259890620107, -0.318575322933718, -0.47550481640187114, -0.36902137183016964, -0.3124678811859988, -0.40073160227343835, -0.41023643948282773, -0.33238989475872815, -0.19301789957946344, -0.22668802672263902, -0.34003204008395854]'. Picking the first and converting the rest.
  X = self.scaler_.transform(X.to_numpy())


#### Eigenvalues

In [131]:
famd.eigenvalues_summary


,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,4.991,8.25%,8.25%
1,3.713,6.14%,14.39%
2,3.099,5.12%,19.51%
3,2.649,4.38%,23.89%
4,2.509,4.15%,28.03%
5,2.505,4.14%,32.17%
6,2.500,4.13%,36.31%
7,2.500,4.13%,40.44%


In [132]:
famd.scree_plot()


alt.Chart(...)

#### Coordinates

In [133]:
famd.row_coordinates(df).head()


/Users/Carmen/.pyenv/versions/3.10.6/envs/game-of-thrones-survival/lib/python3.10/site-packages/prince/pca.py:175: PerformanceWarning: Concatenating sparse arrays with multiple fill values: '[-0.8428710783247098, -1.3377474893729913, -1.4647274544877327, -0.5954607326011404, -1.1608773306309617, -1.0734821019593819, -0.9756806462851751, -1.2442054800010094, -0.2150551448197962, -0.25344491871855485, -0.3602129095201821, -0.3362326801646274, -0.3826934349571483, -0.14778251290408706, -0.42711296887461037, -0.4345675101331315, -0.4195259890620107, -0.318575322933718, -0.47550481640187114, -0.36902137183016964, -0.3124678811859988, -0.40073160227343835, -0.41023643948282773, -0.33238989475872815, -0.19301789957946344, -0.22668802672263902, -0.34003204008395854]'. Picking the first and converting the rest.
  X = self.scaler_.transform(X.to_numpy())


component,0,1,2,3,4,5,6,7
0,92.057507,-35.791547,7.920497,10.701698,-1.427152,3.419896,-0.575832,-1.352882
1,77.378376,-25.645683,8.658154,5.106384,-0.027214,5.192222,-1.030510,-0.314238
2,0.351939,1.880370,-0.893345,-1.081475,-2.482992,1.160129,0.784842,0.468056
3,60.066961,-19.444202,5.896752,8.377050,-0.755938,4.538359,0.158980,2.620386
4,-0.409513,3.718967,1.323006,3.351342,-0.894514,1.442474,1.219074,0.545978


In [134]:
famd.column_coordinates_


component,0,1,2,3,4,5,6,7
variable,,,,,,,,
popularity,0.266500,0.016446,0.011752,0.003611,0.010251,5.028446e-04,1.008992e-08,3.381696e-21
season,0.713286,0.080574,0.002519,0.004414,0.000040,1.968588e-03,5.028568e-06,4.559064e-23
episode_num,0.706042,0.079370,0.002675,0.004209,0.000049,1.952842e-03,5.235840e-06,6.753564e-23
isAlive,0.746327,0.101931,0.003256,0.005105,0.000477,5.630028e-04,2.715688e-10,8.385709e-24
isMarried,0.083620,0.300559,0.231158,0.023233,0.000233,3.752300e-06,2.685844e-09,4.619636e-24
isNoble,0.105037,0.410290,0.168825,0.008035,0.000054,5.831314e-07,7.263362e-08,3.376057e-26
male,0.101648,0.010963,0.267407,0.171410,0.000043,3.032020e-06,1.414536e-08,1.058459e-24
origin,0.285607,0.590926,0.562053,0.846879,0.998564,9.995971e-01,9.999998e-01,1.000000e+00


#### Visualization

In [135]:
famd.plot(
    df,
    x_component=0,
    y_component=1
)


/Users/Carmen/.pyenv/versions/3.10.6/envs/game-of-thrones-survival/lib/python3.10/site-packages/prince/pca.py:175: PerformanceWarning: Concatenating sparse arrays with multiple fill values: '[-0.8428710783247098, -1.3377474893729913, -1.4647274544877327, -0.5954607326011404, -1.1608773306309617, -1.0734821019593819, -0.9756806462851751, -1.2442054800010094, -0.2150551448197962, -0.25344491871855485, -0.3602129095201821, -0.3362326801646274, -0.3826934349571483, -0.14778251290408706, -0.42711296887461037, -0.4345675101331315, -0.4195259890620107, -0.318575322933718, -0.47550481640187114, -0.36902137183016964, -0.3124678811859988, -0.40073160227343835, -0.41023643948282773, -0.33238989475872815, -0.19301789957946344, -0.22668802672263902, -0.34003204008395854]'. Picking the first and converting the rest.
  X = self.scaler_.transform(X.to_numpy())


alt.LayerChart(...)

#### Contributions

In [137]:
(
    famd.row_contributions_
    .sort_values(0, ascending=False)
    .head(5)
    .style.format('{:.3%}')
)


component,0,1,2,3,4,5,6,7
name,,,,,,,,
Daenerys Targaryen,0.544%,0.050%,0.222%,0.112%,0.005%,0.189%,0.002%,0.038%
Daemon Targaryen,0.542%,0.035%,0.067%,0.026%,0.002%,0.189%,0.002%,0.038%
Tywin Lannister,0.465%,0.082%,0.046%,0.044%,0.147%,0.027%,0.013%,0.005%
Aegon Targaryen (son of Aenys I),0.443%,0.020%,0.052%,0.034%,0.000%,0.179%,0.002%,0.038%
Eddard Stark,0.443%,0.025%,0.026%,0.024%,0.012%,0.204%,0.089%,0.002%


In [138]:
famd.column_contributions_.style.format('{:.0%}')


component,0,1,2,3,4,5,6,7
variable,,,,,,,,
popularity,5%,0%,0%,0%,0%,0%,0%,0%
season,14%,2%,0%,0%,0%,0%,0%,0%
episode_num,14%,2%,0%,0%,0%,0%,0%,0%
isAlive,15%,3%,0%,0%,0%,0%,0%,0%
isMarried,2%,8%,7%,1%,0%,0%,0%,0%
isNoble,2%,11%,5%,0%,0%,0%,0%,0%
male,2%,0%,9%,6%,0%,0%,0%,0%
origin,6%,16%,18%,32%,40%,40%,40%,40%
